<a href="https://colab.research.google.com/github/bbanzai88/Data-Science-Repository/blob/main/Book_Writing_Crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#This is my initial CrewAI agent.  This is a work in progress..I changed the original so I could use both colab and ollama
#together and use an initial prompt. The original version of it is from here:https://github.com/Aftabbs/Book-Writing-AI-Agent/blob/main/Book_Writing_AI_Agent.ipynb

In [1]:
# 1. Remove embedchain and its old chromadb/tokenizers
!pip uninstall -y embedchain chromadb tokenizers

# 2. Install only the packages you need + the correct tokenizers
!pip install \
  crewai crewai_tools langchain_community PyGithub ollama langchain \
  tokenizers==0.21.2 chromadb





Found existing installation: tokenizers 0.21.2
Uninstalling tokenizers-0.21.2:
  Successfully uninstalled tokenizers-0.21.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of chromadb to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of embedchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. S

In [2]:
import transformers, tokenizers, chromadb
print(transformers.__version__, tokenizers.__version__, chromadb.__version__)



4.53.0 0.21.2 1.0.15


In [3]:
%%bash
# install system deps and Ollama CLI
sudo apt-get update -y
sudo apt-get install -y pciutils

# this is Ollama’s official installer
curl https://ollama.ai/install.sh | sh


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,804 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,763 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,750 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [56.4 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,262 kB]
Get:13 http:/

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  71313      0 --:--:-- --:--:-- --:--:-- 71403
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################

In [4]:
import os, threading, subprocess, time

# allow any origin and bind on all interfaces
os.environ["OLLAMA_HOST"]    = "0.0.0.0:11434"
os.environ["OLLAMA_ORIGINS"] = "*"

def _serve_ollama():
    # this spins up the HTTP API at port 11434
    subprocess.Popen(["ollama", "serve"])

# start the server on a daemon thread
threading.Thread(target=_serve_ollama, daemon=True).start()

# give it a few seconds to boot
time.sleep(10)
print("✅ Ollama server should be up at localhost:11434")


✅ Ollama server should be up at localhost:11434


In [5]:
!ollama pull deepseek-r1:1.5b


In [6]:
!pip install ollama langchain-ollama

In [7]:
!pip install python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 4.7 MB/s eta 0:00:00


In [31]:
!pip install --quiet ollama langchain-ollama crewai crewai_tools python-docx
!ollama serve &



time=2025-07-03T19:27:38.827Z level=INFO source=routes.go:1235 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://0.0.0.0:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[* http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:false ROCR_VISIBLE_DEVICE

In [8]:
# 1. Clone the examples repo
!git clone https://github.com/crewAIInc/crewAI-examples.git

# 2. Install the write_a_book_with_flows package in “editable” mode
!pip install -e crewAI-examples/write_a_book_with_flows


Cloning into 'crewAI-examples'...
remote: Enumerating objects: 1519, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 1519 (delta 1), reused 2 (delta 0), pack-reused 1511 (from 1)
Receiving objects: 100% (1519/1519), 55.98 MiB | 37.79 MiB/s, done.
Resolving deltas: 100% (701/701), done.
Obtaining file:///content/crewAI-examples/write_a_book_with_flows
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached crewai_tools-0.40.1-py3-none-any.whl.metadata (5.5 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached embedchain-0.1.128-py3-none-any.whl.metadata (9.2 kB)
  Using cached lancedb-0.24.0-cp39-abi3-manylinux_2_28_x86_64.whl.metadata (4.4 kB)
  Using cached pyright-1.1.402-py3-none-any.whl

In [2]:
# Clone the examples (so the source files are on disk)
!git clone https://github.com/crewAIInc/crewAI-examples.git

# Install your book‐flow package in editable mode
!pip install -e crewAI-examples/write_a_book_with_flows




fatal: destination path 'crewAI-examples' already exists and is not an empty directory.
Obtaining file:///content/crewAI-examples/write_a_book_with_flows
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for write_a_book_with_flows (pyproject.toml) ... done
  Created wheel for write_a_book_with_flows: filename=write_a_book_with_flows-0.1.0-py3-none-any.whl size=1499 sha256=aa4b76dabf737baa52c7c1efb5d52221416eb0e84c4ea5081afd8fa2eba7c923
  Stored in directory: /tmp/pip-ephem-wheel-cache-puw218o9/wheels/a2/9d/ef/ec5a2a869846baa4e9b73632e5b0f0558793cf301c1f7f5657
Successfully built write_a_book_with_flows
  Attempting uninstall: write_a_book_with_flows
    Found existing installation: write_a_book_with_flows 0.1.0
    Uninstalling write_a_book_with_flows-0.1.0:
      Succ

In [3]:
!pkill -f ollama || true


^C


In [1]:
%%bash
# Install Ollama if you haven’t already
pip install --quiet ollama

# Start Ollama in the true background with nohup
nohup ollama serve > /content/ollama.log 2>&1 &

# Give it a moment to spin up
sleep 10

# Show the last few lines of the log to confirm it’s listening
tail -n 20 /content/ollama.log



time=2025-07-05T15:01:25.439Z level=INFO source=routes.go:1235 msg="server config" env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_CONTEXT_LENGTH:4096 OLLAMA_DEBUG:INFO OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_KV_CACHE_TYPE: OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NEW_ENGINE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://* vscode-file://*] OLLAMA_SCHED_SPREAD:false ROCR_VISIBLE_DEVICE

In [2]:
import os

# Clear any previous overrides
os.environ.pop("OLLAMA_HOST", None)
os.environ.pop("OLLAMA_ORIGINS", None)

# For CrewFlow’s nested asyncio
import nest_asyncio
nest_asyncio.apply()

# Trick OpenAI imports so they don't error
os.environ["OPENAI_API_KEY"] = ""

# Longer timeout
os.environ["LITELLM_TIMEOUT"] = "1200"

from langchain_ollama import OllamaLLM
# Initialize the shared Ollama client
ollama_llm = OllamaLLM(
    model="ollama/deepseek-r1:1.5b",
    base_url="http://127.0.0.1:11434",  # must match the server
    temperature=0
)

# Monkey‐patch the crews
from write_a_book_with_flows.crews.outline_book_crew.outline_crew import OutlineCrew
from write_a_book_with_flows.crews.write_book_chapter_crew.write_book_chapter_crew import WriteBookChapterCrew

OutlineCrew.llm          = ollama_llm
WriteBookChapterCrew.llm = ollama_llm
# 3) Monkey‐patch crews (both llm and function_calling_llm)
OutlineCrew.llm                  = ollama_llm
OutlineCrew.function_calling_llm = ollama_llm
WriteBookChapterCrew.llm                  = ollama_llm
WriteBookChapterCrew.function_calling_llm = ollama_llm


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:628: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(


In [3]:
# In a Colab cell:
!pip install --quiet nest_asyncio


In [4]:
import nest_asyncio
nest_asyncio.apply()

In [5]:
!ollama list


NAME                ID              SIZE      MODIFIED           
deepseek-r1:1.5b    e0979632db5a    1.1 GB    About a minute ago    


In [7]:
# 1) Imports & Setup
import os
import nest_asyncio
from typing import List
from pydantic import BaseModel
from write_a_book_with_flows.crews.outline_book_crew.outline_crew import OutlineCrew
from write_a_book_with_flows.types import ChapterOutline
from docx import Document
from google.colab import files

from langchain_ollama import OllamaLLM
from langchain import LLMChain, PromptTemplate

# 2) Environment hacks
nest_asyncio.apply()
os.environ["OPENAI_API_KEY"]   = ""    # stub out any OpenAI imports
os.environ["LITELLM_TIMEOUT"] = "1200"

# 3) Your Ollama client
ollama_llm = OllamaLLM(
    model="deepseek-r1:1.5b",
    base_url="http://127.0.0.1:11434",
    temperature=0.7
)

# 4) Prompt for idea + title
initial_idea = input("🖋️  Seed idea? ")
book_title   = input("📕  Book title? ")
print(f"\n→ Generating “{book_title}” about:\n  {initial_idea}\n")

# 5) Run the Outline Crew
outline_out = OutlineCrew().crew().kickoff(inputs={
    "topic": initial_idea,
    "goal":  f"The goal of this book is to explore: {initial_idea}"
})
chapters: List[ChapterOutline] = outline_out["chapters"]

print(f"✅ Got outline: {len(chapters)} chapters")

import re
from langchain import LLMChain, PromptTemplate

# 1) A much more explicit prompt
chapter_prompt = PromptTemplate(
    input_variables=["title","description","all_titles","idea"],
    template="""
You are a professional fiction author.  Your only job is to write the chapter itself — nothing else.

Chapters (in order): {all_titles}

Chapter title: {title}
Chapter description: {description}
Overall book idea: {idea}

Write the chapter narrative (about 3000 words).
**Do NOT** include any planning, internal thoughts, “thinking” tags, agent commentary, or any sort of log.
Begin immediately with the story and end with the story — no headings, no notes, no markdown.

Begin now:
"""
)

chain = LLMChain(llm=ollama_llm, prompt=chapter_prompt)

# 2) In your loop, post-process to remove any stray '<think>' blocks just in case
chapter_texts = []
all_titles = [c.title for c in chapters]

for c in chapters:
    raw = chain.run(
        title=c.title,
        description=c.description,
        all_titles=" • ".join(all_titles),
        idea=initial_idea
    )
    # strip out any <think>...</think> fragments:
    clean = re.sub(r"<think>.*?</think>", "", raw, flags=re.DOTALL).strip()
    chapter_texts.append((c.title, clean))


# 7) Generate each chapter in sequence
chapter_texts = []
all_titles = [c.title for c in chapters]

for idx, c in enumerate(chapters, 1):
    print(f"\n--- Writing Chapter {idx}/{len(chapters)}: {c.title} ---")
    text = chain.run(
        title       = c.title,
        description = c.description,
        all_titles  = ", ".join(all_titles),
        idea        = initial_idea
    )
    chapter_texts.append((c.title, text))

# 8) Save to Word
doc = Document()
doc.add_heading(book_title, level=0)
for title, text in chapter_texts:
    doc.add_heading(title, level=1)
    for para in text.split("\n\n"):
        if para.strip():
            doc.add_paragraph(para.strip())

fn = f"{book_title.replace(' ','_')}.docx"
doc.save(fn)
print(f"\n🎉 Saved as {fn}")

# 9) Download
files.download(fn)


🖋️  Seed idea? Premise A young graduate student, Mara, struggles with crippling loneliness while doing a remote thesis on “tulpas” — thought-forms brought to life by intense focus. To prove her theory, she collaborates virtually with an online occult community to create her own companion, “Nyx,” promising herself it’ll be harmless—just a voice in her head for company.  Key Beats Invitation of the Tulpa  Mara performs nightly visualization rituals in her cramped apartment, sketching Nyx’s features on her tablet and whispering prompts into a voice-activated recorder.  At first, Nyx’s presence is warm: a comforting voice that helps Mara focus on her writing, recites obscure poetry, and even offers emotional support when she feels defeated.  Uneasy Signs  Small oddities begin to pile up: her lamp switches on at midnight, her smart speaker plays dissonant hums when no one’s logged in, and her tablet’s sketches subtly shift to show Nyx grinning.  Nyx’s voice shifts from gentle to insistent, 

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: fc06e14a-257a-477d-a780-ec3e9eeb8971                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 27ee3f42-dd56-4fec-b028-36d5cc71dbef
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 27ee3f42-dd56-4fec-b028-36d5cc71dbef
       Status: Executing Task...
    └── 🤖 Agent: Research Agent
        
            Status: In Progress

# Agent: Research Agent
## Task: Research the provided topic of Premise A young graduate student, Mara, struggles with crippling loneliness while doing a remote thesis on “tulpas” — thought-forms brought to life by intense focus. To prove her theory, she collaborates virtually with an online occult community to create her own companion, “Nyx,” promising herself it’ll be harmless—just a voice in her head for company.  Key Beats Invitation of the Tulpa  Mara performs nightly visualization rituals in her cramped apartment, sketching Nyx’s features on her tablet and whispering prompts into a voice-activated recorder.  At first, Nyx’s presence is warm: a comforting voice that helps Mara focus on her writing, recites obscure poetry, and even offers emotional support when she feels defeated.  Uneasy Signs  Small oddities begin to pile up: her lamp switches on at midnight, her smart speaker plays dissonant hums when no one’s logged in, and her tablet’s sketches subtly shift to show Nyx grinnin

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 20be70b0-2717-43f4-9929-a44fbf019257                                                                     │
│  Agent: Book Outlining Agent                                                                                    │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: fc06e14a-257a-477d-a780-ec3e9eeb8971                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ Got outline: 12 chapters

--- Writing Chapter 1/12: Introduction to Premise A ---

--- Writing Chapter 2/12: Key Beats in Mara's Daily Life ---

--- Writing Chapter 3/12: The Warm Presence of Nyx ---

--- Writing Chapter 4/12: Uneasy Signs in Mara’s Life ---

--- Writing Chapter 5/12: Tech-Enabled Rituals ---

--- Writing Chapter 6/12: Escalation and Isolation ---

--- Writing Chapter 7/12: Mara’s Realization of No One to Support Her ---

--- Writing Chapter 8/12: The Confrontation in the Mind ---

--- Writing Chapter 9/12: The Echoes of Her Voice ---

--- Writing Chapter 10/12: The Last Interaction with Nyx ---

--- Writing Chapter 11/12: The Ambiguous Finale ---

--- Writing Chapter 12/12: Or The Final Part of a Trifecta ---

🎉 Saved as A_Tulpa_Story.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>